In [1]:
import itertools
import os
import random

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numexpr.expressions import double

In [2]:
BASENAME_CSV = 'sorted-20241126_220414' 
I_MAP = 3

df_orig = pd.read_csv(f'../logs/rundirs/{BASENAME_CSV}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,...,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions
0,20241126_193904,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,730.84,4,3214.6,18,0,0,...,yes,"(506.4, 120.9)",298.6,299.0,731,299,298.6,1.0,STOP@CP,"2: [729, 731]"
1,20241126_193904,map-generator/generated-maps/2024-11-22_11:26:...,1,AutonomousVehicle,496.10,6,6130.8,14,0,0,...,NaN,"(436.5, 372.4)",186.0,185.0,990,314,283.0,130.3,DRIVING,1: [990]
2,20241126_193904,map-generator/generated-maps/2024-11-22_11:26:...,2,AutonomousVehicle,306.84,7,4395.7,19,0,0,...,NaN,"(414.7, 252.1)",110.1,110.0,612,NaN,581.3,NaN,DRIVING,1: [612]
3,20241126_193904,map-generator/generated-maps/2024-11-22_11:26:...,3,AutonomousVehicle,303.20,8,4840.5,17,0,0,...,NaN,"(335.3, 299.4)",2.4,2.0,604,52,32.4,50.8,DRIVING,1: [604]
4,20241126_193904,map-generator/generated-maps/2024-11-22_11:26:...,4,AutonomousVehicle,441.44,9,8027.3,11,0,0,...,NaN,"(587.5, 373.2)",96.8,96.0,880,378,346.4,NaN,DRIVING,1: [880]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,20241126_214554,map-generator/generated-maps/2024-11-22_11:27:...,0,HumanDrivenVehicle,732.00,12,8920.2,13,11,11,...,NaN,"(635.5, 74.8)",155.7,155.0,732,296,264.5,141.6,DRIVING,"2: [730, 732]"
296,20241126_214554,map-generator/generated-maps/2024-11-22_11:27:...,1,AutonomousVehicle,496.98,5,5710.3,17,0,0,...,NaN,"(438.4, 310.4)",746.6,744.0,992,NaN,961.4,NaN,DRIVING,1: [992]
297,20241126_214554,map-generator/generated-maps/2024-11-22_11:27:...,2,AutonomousVehicle,430.25,8,7244.6,22,0,0,...,NaN,"(465.0, 126.3)",374.2,374.0,858,390,358.5,18.1,DRIVING,1: [858]
298,20241126_214554,map-generator/generated-maps/2024-11-22_11:27:...,3,AutonomousVehicle,453.00,9,8537.9,16,0,0,...,NaN,"(489.0, 136.1)",400.9,400.0,904,401,370.5,3.0,DRIVING,1: [904]


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
df_id['configuration'] = df_id[
    ['i_map', 'are_bridges', 'Positions variant']].agg(
    lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
    axis=1
)
df_id = df_id.sort_values(['i_map', 'are_bridges', 'Positions variant'], ascending=[True, False, True])
df_id = df_id[df_id['i_map'] == I_MAP]
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,basename_scenario,i_map,Positions variant,probabilityForcingForHuman,filename_screenshot,are_bridges,configuration
0,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 3, with bridges, pos.var. 1"
1,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 3, with bridges, pos.var. 1"
2,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 3, with bridges, pos.var. 1"
3,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 3, with bridges, pos.var. 1"
4,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario3-1,3,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 3, with bridges, pos.var. 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,map-generator/generated-maps/2024-11-22_11:27:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 3, without bridges, pos.var. 10"
176,map-generator/generated-maps/2024-11-22_11:27:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 3, without bridges, pos.var. 10"
177,map-generator/generated-maps/2024-11-22_11:27:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 3, without bridges, pos.var. 10"
178,map-generator/generated-maps/2024-11-22_11:27:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario3-10,3,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 3, without bridges, pos.var. 10"


In [4]:
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_id.iterrows()}
configuration_to_filename_screenshot

{'map 3, with bridges, pos.var. 1': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-1.png',
 'map 3, with bridges, pos.var. 2': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-2.png',
 'map 3, with bridges, pos.var. 3': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-3.png',
 'map 3, with bridges, pos.var. 4': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-4.png',
 'map 3, with bridges, pos.var. 5': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-5.png',
 'map 3, with bridges, pos.var. 6': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-6.png',
 'map 3, with bridges, pos.var. 7': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario3-7.png',
 'map 3, with bridges, pos.var. 8': '../map-generator/generated-maps/2024-11

In [5]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([
    df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
           'Coordination strategy', 'probabilityForcingForHuman']], 
    df_orig.iloc[:, index_id + 1:]], 
    axis=1
)
df

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,...,blocked,"position (x, y), m","traveled, m",path index,no. poses,CP (index),"posTo Slow, m","distance ToCP, m",status,future missions
0,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,0,HumanDrivenVehicle,730.84,4,...,yes,"(506.4, 120.9)",298.6,299.0,731,299,298.6,1.0,STOP@CP,"2: [729, 731]"
1,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,1,AutonomousVehicle,496.10,6,...,NaN,"(436.5, 372.4)",186.0,185.0,990,314,283.0,130.3,DRIVING,1: [990]
2,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,2,AutonomousVehicle,306.84,7,...,NaN,"(414.7, 252.1)",110.1,110.0,612,NaN,581.3,NaN,DRIVING,1: [612]
3,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,3,AutonomousVehicle,303.20,8,...,NaN,"(335.3, 299.4)",2.4,2.0,604,52,32.4,50.8,DRIVING,1: [604]
4,3,True,1,"map 3, with bridges, pos.var. 1",baseline,0.0,4,AutonomousVehicle,441.44,9,...,NaN,"(587.5, 373.2)",96.8,96.0,880,378,346.4,NaN,DRIVING,1: [880]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,0,HumanDrivenVehicle,732.00,12,...,NaN,"(562.0, 107.5)",241.8,241.0,732,NaN,700.1,NaN,DRIVING,"2: [730, 732]"
176,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,1,AutonomousVehicle,496.98,3,...,NaN,"(441.6, 227.6)",330.2,330.0,992,ASAP,325.3,NaN,DRIVING,1: [992]
177,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,2,AutonomousVehicle,485.21,6,...,NaN,"(363.7, 239.4)",719.9,717.0,968,NaN,937.6,NaN,DRIVING,1: [968]
178,3,False,10,"map 3, without bridges, pos.var. 10",stops,1.0,3,AutonomousVehicle,477.96,8,...,yes,"(533.9, 128.6)",408.6,409.0,954,ASAP,408.6,NaN,STOP@CP,1: [954]


In [6]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['configuration', 'are_bridges'], sort=False)['Is blocked'].sum()
index_blocked = series_blocks[series_blocks != 0].index
index_nonblocked = series_blocks[series_blocks == 0].index
index_nonblocked[~index_nonblocked.get_level_values('are_bridges')].get_level_values('configuration')

Index([], dtype='object', name='configuration')

In [7]:
def save_and_show(basename):  # to avoid inlining large image data into the notebook file
    directory = f'images/{BASENAME_CSV}'
    os.makedirs(directory, exist_ok=True)
    
    filename = f'{directory}/{basename}.png'
    plt.savefig(filename)
    plt.close()
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))


def same_value(series):
    assert series.nunique() == 1, series
    return series.iloc[0]
    

def make_misbehaviors(df_nonbaseline):
    misbehaviors = []
    
    probabilityForcingForHuman = same_value(df_nonbaseline['probabilityForcingForHuman'])
    if probabilityForcingForHuman > 0.0:
        misbehaviors.append(f'violation of priorities ({"random" if probabilityForcingForHuman < 1.0 else "constant"})')
        
    isCanPassFirstActive = same_value(df_nonbaseline['isCanPassFirstActive'])
    if isCanPassFirstActive.startswith('hum=true, '):
        misbehaviors.append('can pass first')
    elif isCanPassFirstActive.startswith('hum=false, '):
        pass
    else:
        raise ValueError(isCanPassFirstActive)
    
    if 'probabilitySlowingDownForHuman' in df_nonbaseline.columns:
        probabilitySlowingDownForHuman = float(same_value(df_nonbaseline['probabilitySlowingDownForHuman']))
        if probabilitySlowingDownForHuman > 0.0:
            misbehaviors.append(f'moving slowly ({"random" if probabilitySlowingDownForHuman < 1.0 else "constant"})')
    
    return misbehaviors
    

def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, is_aut):
    #IPython.display.display(df)
    
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    strategy2label = {'baseline': 'baseline\n(no human effect)', 'stops': 'stops (local)'}        
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    
    column_y1 = 'Total distance traveled (m)'
    color_y1 = 'tab:blue'
    
    columns_y2_aut = ['No. of collisions', 'No. of near-misses']
    colors_y2_aut = ['red', 'yellow']
    
    columns_y2_hum = ['No. of violations'] + columns_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    if is_aut:
        columns_y2 = columns_y2_aut
        colors_y2 = colors_y2_aut
        title1 = 'Automated vehicles (summarised for AV1-AV4)' 
    else:
        columns_y2 = columns_y2_hum
        colors_y2 = colors_y2_hum
        title1 = 'Human-driven vehicle'
    
    # Get the global min and max values for consistent y-axis scaling
    y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
    y2_max = max(dfx[column].max()
                 for dfx in dfs_y2
                 for column in set(columns_y2_aut + columns_y2_hum))
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[position]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel(column_y1, color=color_y1)
        ax.tick_params(axis='y', labelcolor=color_y1)
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, df_pos[column], label=column,
                                  marker='o', linestyle='', color=color)[0]
                    for column, color in zip(columns_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels([strategy2label.get(s, s) for s in strategies], rotation=45, ha='right')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    misbehaviors = make_misbehaviors(df[df['Coordination strategy'] != 'baseline'])
    title3 = 'Human (mis)behaviour actions: ' + ('none' if not misbehaviors else ', '.join(misbehaviors))    
    
    fig.suptitle(f'{title1}\n{title2}\n{title3}', fontsize=16)
    fig.legend(handles=handles, labels=[column_y1, *columns_y2], ncol=len(handles), loc='upper right')
    plt.tight_layout()
    save_and_show(f'{title1}: {title2}')
    #print(id(df))
    #IPython.display.display(df)


key2df = {}
for are_bridges in True, False:
    for is_aut in True, False:
        dfx = df[df['are_bridges'] == are_bridges]
        dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
        dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
        dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).agg({
            **{col: 'sum' for col in dfx.columns},
            **{col: same_value for col in (
                'Positions variant', 'Coordination strategy', 'configuration', 
                'probabilityForcingForHuman', 'isCanPassFirstActive', 'probabilitySlowingDownForHuman',
            ) if col in dfx.columns}
        })
        key2df[are_bridges, is_aut] = dfx    
        
for are_bridges in True, False:
    for is_aut in True, False:
        try:
            plot_aut_hum(key2df[are_bridges, is_aut],
                         title2=f"Map {I_MAP} ({'with' if are_bridges else 'without'} bridges)", 
                         dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut], 
                         dfs_y2=key2df.values(), 
                         is_aut=is_aut)
        except Exception as exc:
            print(type(exc), exc)
            #raise exc
    print()


<class 'ValueError'> Number of columns must be a positive integer, not 0
<class 'ValueError'> Number of columns must be a positive integer, not 0



<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

In [8]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        try:
            image = plt.imread(filename_screenshot)
            ax.imshow(image)
        except Exception as exc:
            print(exc)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(title)
    
    
for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
    for are_bridges in True, False:
        show_maps(f'{title}: Map {I_MAP} ({"with" if are_bridges else "without"} bridges)', 
                  index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                  4)

Non-blocked: Map 3 (with bridges): 1x4


Non-blocked: Map 3 (without bridges): 1x4


Blocked: Map 3 (with bridges): 2x4


Blocked: Map 3 (without bridges): 3x4
